# NN Model
#### cv score :  0.5468884688090737 (5 folds)
#### cv score :  ?? (10 folds)

In [ ]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import sklearn
from Functions import prepro, NNModel
import warnings

import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow.keras.callbacks import EarlyStopping


random.seed(0)

pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)
warnings.filterwarnings('ignore')

In [ ]:
train_df = pd.read_csv("../../data/Processed/train2.csv")
test_df = pd.read_csv("../../data/Processed/test2.csv")
print(train_df.shape)
print(test_df.shape)

In [ ]:
y = train_df["y"]
train_df = train_df.drop("y", axis=1)

In [ ]:
# add disconnection info

train_df = prepro.add_disconnection(train_df)
test_df = prepro.add_disconnection(test_df)

In [ ]:
# add info about numeric column
# 武器データの中のnum_colsを持つ特徴について、チーム内の統計量を計算、levelは結構効いてる

# num_cols = ["level", "range", "rapid", "atack", "ink-sub", "fav-main", "good-special"]

num_cols = [
    "level", "range-main", "range-bullet-main", "distant-range_sub", 
    "rapid", "atack", "ink-sub", "fav-main", "good-special", "DPS", "kill_time_ika-main",
    "front_gap_human-main", "front_gap_ika-main", "rensya_frame-main", "saidai_damege-main", "damage_min-sub", 
    "damage_max-sub", "install_num-sub", "good-sub", "damage_max-special", 
    "duration-special", "good-special", "direct_rad-special", "distant_rad-special"
]

cols = [col for col in train_df.columns if "A1" in col or "A2" in col or "A3" in col or "A4" in col or 
        "B1" in col or "B2" in col or "B3" in col or "B4" in col]
drop_cols = []
for col1 in cols:
    f = True
    for col2 in num_cols:
        if col2 in col1:
            f = False
    if f and train_df[col1].dtype in [int, float]:
        drop_cols.append(col1)
        
#train_df = train_df.drop(columns=drop_cols)
#test_df = test_df.drop(columns=drop_cols)
        
    

train_df = prepro.add_numeric_info(train_df, num_cols)
test_df = prepro.add_numeric_info(test_df, num_cols)

train_df["level-mean-sub"] = train_df["A-level-mean"] - train_df["B-level-mean"]
train_df["level-mean-div"] = train_df["A-level-mean"] / train_df["B-level-mean"]
train_df["level-median-sub"] = train_df["A-level-median"] - train_df["B-level-median"]
train_df["level-median-div"] = train_df["A-level-median"] / train_df["B-level-median"]
test_df["level-mean-sub"] = test_df["A-level-mean"] - test_df["B-level-mean"]
test_df["level-mean-div"] = test_df["A-level-mean"] / test_df["B-level-mean"]
test_df["level-median-sub"] = test_df["A-level-median"] - test_df["B-level-median"]
test_df["level-median-div"] = test_df["A-level-median"] / test_df["B-level-median"]

# 欠損値埋め先にやろう
欠損値がある列

rank, weapon, level ⇨　回線切れ or ナワバリ

weaponの欠損に並んでそれに関係ある列、A3, A4, B3, B4 ⇨ 回線切れ

level, weaponが消えていたら回線切れ

In [ ]:
# rankの欠損値を埋める
train_df, test_df = prepro.fillna_rank(train_df, test_df)

#そのほかの欠損値を埋める
train_df, test_df = prepro.fillna(train_df, test_df)

In [ ]:
print(train_df.isnull().sum().sum())
print(test_df.isnull().sum().sum())

# 特徴量エンジニアリング

In [ ]:
# count reskin, by mode
print("reskin")
train_df, test_df = prepro.count_reskin(train_df, test_df)
train_df, test_df = prepro.count_reskin_by_mode(train_df, test_df)

# count mainweapon, by mode
print("mainweapon")
train_df, test_df = prepro.count_mainweapon(train_df, test_df)
train_df, test_df = prepro.count_mainweapon_by_mode(train_df, test_df)

# count subweapon, by mode
print("subweapon")
train_df, test_df = prepro.count_subweapon(train_df, test_df)
train_df, test_df = prepro.count_subweapon_by_mode(train_df, test_df)

# count special, by mode
print("special")
train_df, test_df = prepro.count_special(train_df, test_df)
train_df, test_df = prepro.count_special_by_mode(train_df, test_df)


#identify A1
train_df, test_df = prepro.identify_A1(train_df, test_df)


# 水増し, A1も統計量に含めた特徴を作る場合は水ましより先にやる
print("mizumashi")
train_df, y = prepro.mizumashi(train_df, y)

# is_nawabari
train_df, test_df = prepro.is_nawabari(train_df, test_df)

# match rank、単体で意味なし
train_df, test_df = prepro.match_rank(train_df, test_df)

# rankを二列に分ける
train_df, test_df = prepro.ranker(train_df, test_df)

In [ ]:
# add team info、メインはなくてもいい

train_df,  test_df = prepro.addTeamInfo(train_df, test_df, cols=["special", "subweapon", "category1", "category2", "mainweapon"])
#train_df,  test_df = prepro.addTeamInfo(train_df, test_df, cols=["special", "subweapon", "category1", "category2"])

In [ ]:
# categorize team , 良い

categorize_col = ["category1", "category2", "subweapon", "special", "mainweapon"]
#categorize_col = ["category1", "category2", "subweapon", "special"]
for col in categorize_col:
    print(col)
    train_df, test_df = prepro.categorize_team(train_df, test_df, col)

In [ ]:
# product categorical feature
train_df, test_df = prepro.prod(train_df, test_df, "mode", "stage")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category1-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category1-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category2-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category2-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-mainweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-mainweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-subweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-subweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-special-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-special-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "match_rank")

train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category1-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category1-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category2-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category2-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-mainweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-mainweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-subweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-subweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-special-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-special-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "match_rank")


train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A1")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A2")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A3")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A4")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B1")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B2")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B3")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B4")

# Make Input

In [ ]:
# プレイヤー固有のカテゴリ変数は個別にエンコードしなければいけないね…
# rank
# category1
# category2
# subweapon
# special
# mainweapon
# reskin
# mode x reskin
# make input


drop_cols = ["id", "lobby", "lobby-mode",  "period", "game-ver", "A1-weapon", "A2-weapon", "A3-weapon", "A4-weapon", \
              "B1-weapon", "B2-weapon", "B3-weapon", "B4-weapon"]

categorical_feature = [col for col in train_df.dtypes[train_df.dtypes == "object"].index.to_list() if col not in drop_cols]

from sklearn.preprocessing import LabelEncoder
for col in ["category1", "category2", "subweapon", "special", "mainweapon", "reskin", "mode x reskin"]:
    lbl = LabelEncoder()
    lbl.fit(train_df[col+"-A4"])
    train_df[col+"-A1"] = lbl.transform(train_df[col+"-A1"])
    train_df[col+"-A2"] = lbl.transform(train_df[col+"-A2"])
    train_df[col+"-A3"] = lbl.transform(train_df[col+"-A3"])
    train_df[col+"-A4"] = lbl.transform(train_df[col+"-A4"])
    train_df[col+"-B1"] = lbl.transform(train_df[col+"-B1"])
    train_df[col+"-B2"] = lbl.transform(train_df[col+"-B2"])
    train_df[col+"-B3"] = lbl.transform(train_df[col+"-B3"])
    train_df[col+"-B4"] = lbl.transform(train_df[col+"-B4"])
    
    test_df[col+"-A1"] = lbl.transform(test_df[col+"-A1"])
    test_df[col+"-A2"] = lbl.transform(test_df[col+"-A2"])
    test_df[col+"-A3"] = lbl.transform(test_df[col+"-A3"])
    test_df[col+"-A4"] = lbl.transform(test_df[col+"-A4"])
    test_df[col+"-B1"] = lbl.transform(test_df[col+"-B1"])
    test_df[col+"-B2"] = lbl.transform(test_df[col+"-B2"])
    test_df[col+"-B3"] = lbl.transform(test_df[col+"-B3"])
    test_df[col+"-B4"] = lbl.transform(test_df[col+"-B4"])
    
for col in ["rank-mark", "rank"]:
    lbl = LabelEncoder()
    lbl.fit(train_df["A4-" + col])
    train_df["A1-"+col] = lbl.transform(train_df["A1-"+col])
    train_df["A2-"+col] = lbl.transform(train_df["A2-"+col])
    train_df["A3-"+col] = lbl.transform(train_df["A3-"+col])
    train_df["A4-"+col] = lbl.transform(train_df["A4-"+col])
    train_df["B1-"+col] = lbl.transform(train_df["B1-"+col])
    train_df["B2-"+col] = lbl.transform(train_df["B2-"+col])
    train_df["B3-"+col] = lbl.transform(train_df["B3-"+col])
    train_df["B4-"+col] = lbl.transform(train_df["B4-"+col])

    test_df["A1-"+col] = lbl.transform(test_df["A1-"+col])
    test_df["A2-"+col] = lbl.transform(test_df["A2-"+col])
    test_df["A3-"+col] = lbl.transform(test_df["A3-"+col])
    test_df["A4-"+col] = lbl.transform(test_df["A4-"+col])
    test_df["B1-"+col] = lbl.transform(test_df["B1-"+col])
    test_df["B2-"+col] = lbl.transform(test_df["B2-"+col])
    test_df["B3-"+col] = lbl.transform(test_df["B3-"+col])
    test_df["B4-"+col] = lbl.transform(test_df["B4-"+col])

In [ ]:
# make input

X, test_X = prepro.make_input(train_df, test_df, drop_cols, categorical_encode=True, scaler=True, verbose=False)



In [ ]:
X

In [ ]:
# 全データを5つに分割
random.seed(random.randint(0, 10000))
SIZE = X.shape[0]
K = 5
cat_tgtenc_cols = ["mode", "stage", "team-category1-A", "team-category1-B"]
#cat_tgtenc_cols = ["mode", "stage"]

folds = prepro.make_kfolds(SIZE, K)
print(len(folds))
for i, fold in enumerate(folds):
    print("fold ", i+1, " size is ", len(fold))
    
    
if SIZE != len(set(sum(folds, []))):
    print("error is occuring in spliting")
else :
    print("successfully split")

In [ ]:
THRESHOLD = 0.50
models = []
cv_scores = []
temp = 0
train_pred = []

all_indices = sum(folds, [])
for i in range(K):
    print("======================== fold {} ========================".format(i+1))
    valid_indices = folds[i]
    train_indices = list(set(all_indices) - set(valid_indices))
    # print("train ", len(train_indices), " , valid ", len(valid_indices))
    train_X = X.iloc[train_indices]
    train_y = y.iloc[train_indices]
    valid_X = X.iloc[valid_indices]
    valid_y = y.iloc[valid_indices]
    
    
    model = NNModel.make_model(X, test_X, categorical_feature)
    #model = make_model(X, test_X, categorical_feature)
    
    
    train_X = NNModel.prepro_nn(train_X, categorical_feature)
    train_y = train_y.values.reshape(-1, 1)
    valid_X = NNModel.prepro_nn(valid_X, categorical_feature)
    valid_y = valid_y.values.reshape(-1, 1)
    
    
    

    cb = EarlyStopping(monitor='val_loss', patience=1)

    model.fit(
        train_X, 
        train_y,
        batch_size=64,
        epochs=10,
        verbose=1,
        callbacks=[cb],
        validation_data=(valid_X, valid_y),
    )
    
    pred = model.predict(valid_X)
    pred = np.where(pred < THRESHOLD, 0, 1)
    train_pred.append(pred)
    temp += np.sum(pred)
    
    score = accuracy_score(pred, valid_y)
    
    models.append(model)
    cv_scores.append(score)
    
    
print("cv score : ", np.mean(cv_scores))    
print("cv ratio : ", temp / SIZE)

In [ ]:
preds = []
test_X = NNModel.prepro_nn(test_X, categorical_feature)

for i in range(K):
    model = models[i]
    pred = model.predict(test_X, batch_size=128).reshape(-1, )
    preds.append(pred)
    print(np.sum(pred) / pred.shape[0])
    
preds = np.array(preds)
preds = np.mean(preds, axis=0)
print(np.sum(preds) / preds.shape[0])


temp = pd.DataFrame({"pred":preds})
temp.hist(bins=20)

In [ ]:
preds = np.where(preds < THRESHOLD, 0, 1)
print(np.sum(preds) / preds.shape[0])

submit_df = pd.DataFrame({'y': preds})
submit_df.index.name = 'id'
submit_df.to_csv('../Submissions/submission_NN_2_{}.csv'.format(K))

# モデル解釈

In [ ]:
importance = pd.DataFrame(models[1].feature_importance(), index=X.columns, columns=['importance']).sort_values('importance', ascending=False)
display(importance[:50])

In [ ]:
importance["importance"].sum()

In [ ]:
train_df["pred"] = 0
train_df["y"] = y.values
for i in range(K):
    train_df["pred"].iloc[folds[i]] = train_pred[i].reshape(-1, )

In [ ]:
result_df = train_df[train_df["y"] != train_df["pred"]]

In [ ]:
print(result_df[result_df["mode"] == "nawabari"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "hoko"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "asari"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "area"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "yagura"].shape[0]/result_df.shape[0])

In [ ]:
result_df